- Проверить наличие в справочниках адресов, брендов, граммовок, регионов (если нет добавить)
- Взять файл
- Заменить адреса, субъекты, бренды, вес на корректные
- Объединить по месяцу, году, региону, бренду, адресу, сумма по весу
- то же по конкурентам
- объединить их вместе
- выгрузить в файл
- убрать адрес Санкт Петербург, Пироговская наб., д.17
- сделать сводный

In [1]:
import pandas as pd

from datetime import datetime

import xlrd

### импорт нужных таблиц

In [2]:
path = 'D:\\Analysis Burn'

In [3]:
burn = pd.read_csv(path + '\\Википедия\\БЁРН исходники\\burn_tobacco.csv')
dsmh = pd.read_csv(path + '\\Википедия\\СВМХ исходники\\dsmh.csv')

In [56]:
addresses = pd.read_excel(path + '\\Википедия\\Справочники\\addresses.xlsx')

In [57]:
regions = pd.read_excel(path + '\\Википедия\\Справочники\\regions.xlsx')

In [58]:
brands = pd.read_excel(path + '\\Википедия\\Справочники\\brands.xlsx')

In [59]:
weight = pd.read_excel(path + '\\Википедия\\Справочники\\weight.xlsx')

In [60]:
burn_products_price = pd.read_excel(r"D:\Analysis Burn\Википедия\Справочники\burn_products_price.xlsx")

In [61]:
dsmh_products_price = pd.read_excel(r"D:\Analysis Burn\Википедия\Справочники\dsmh_products_price.xlsx")

# Получение отсутствующих значений в справочниках

In [62]:
addresses_bad = pd.concat([dsmh['адрес'], burn['Адрес']] )

In [63]:
addresses_bad = addresses_bad.drop_duplicates()
addresses_bad = addresses_bad.dropna()

In [64]:
addresses_bad = pd.DataFrame(addresses_bad)

In [65]:
addresses_bad = addresses_bad.rename(columns={0:'Адрес'})

In [66]:
merged = pd.merge(addresses, addresses_bad, left_on='Исходный адрес', right_on = 'Адрес', how='outer', indicator=True)

In [67]:
missing_values = merged.loc[ merged['_merge'] == 'right_only' ]

In [68]:
if missing_values.empty == False:
    missing_values.to_excel(r'C:\Users\User\Desktop\missing_addresses.xlsx', index=False)
else:
    print("Он пустой")

Он пустой


In [69]:
regions_bad = pd.concat([dsmh['рег'], burn['Субъект РФ']] )

In [70]:
regions_bad = regions_bad.drop_duplicates()
regions_bad = regions_bad.dropna()

In [71]:
regions_bad = pd.DataFrame(regions_bad)

In [72]:
regions_bad =regions_bad.rename(columns={0:'Регион'})

In [73]:
merged2 = pd.merge(regions, regions_bad, left_on='Субъект РФ исходный', right_on = 'Регион', how='outer', indicator=True)

In [74]:
missing_values2 = merged2.loc[merged2['_merge'] == 'right_only', 'Регион']

In [75]:
if missing_values2.empty == False:
    missing_values2.to_excel(r'C:\Users\User\Desktop\missing_regions.xlsx')
else:
    print("Он пустой")

In [76]:
brands_bad = pd.concat([dsmh['бренд'], burn['Наименование товара']] )

In [77]:
brands_bad = brands_bad.drop_duplicates()
brands_bad = brands_bad.dropna()

In [78]:
brands_bad = pd.DataFrame(brands_bad)

In [79]:
brands_bad = brands_bad.rename(columns={0:'Бренд'})

In [80]:
merged3 = pd.merge(brands, brands_bad, left_on='Бренд (исходный)', right_on = 'Бренд', how='outer', indicator=True)

In [81]:
missing_values3 = merged3.loc[merged3['_merge'] == 'right_only', 'Бренд_y']

In [82]:
if missing_values3.empty == False:
    missing_values3.to_excel(r'C:\Users\User\Desktop\missing_brands.xlsx')
else:
    print("Он пустой")

Он пустой


In [83]:
weight_bad = pd.concat([dsmh['вес'], burn['Наименование товара']] )

In [84]:
weight_bad = weight_bad.drop_duplicates()

In [85]:
weight_bad = pd.DataFrame(weight_bad)

In [86]:
weight_bad = weight_bad.rename(columns={0:'Граммовка'})

In [87]:
merged4 = pd.merge(weight, weight_bad, left_on='Исходная граммовка', right_on = 'Граммовка', how='outer', indicator=True)

In [88]:
missing_values4 = merged4.loc[merged4['_merge'] == 'right_only', 'Граммовка_y']

In [89]:
if missing_values4.empty == False:
    missing_values4.to_excel(r'C:\Users\User\Desktop\missing_weight.xlsx')
else:
    print("Он пустой")

Он пустой


### Проверка наличия цен

In [90]:
dsmh_products_price['Наименование товара'] = dsmh_products_price['продукт']+dsmh_products_price['вес'].astype(str)
dsmh_products_price =dsmh_products_price[['Наименование товара', 'Цена']]

In [91]:
prices = pd.concat([dsmh_products_price, burn_products_price] )

In [92]:
prices = prices.drop_duplicates()
prices = prices.dropna()

In [93]:
prices = pd.DataFrame(prices)

In [94]:
burn_product = pd.DataFrame()
burn_product['Наименование товара'] = burn['Наименование товара']

In [95]:
dsmh_product = pd.DataFrame()
dsmh_product['Наименование товара'] = dsmh['продукт']+dsmh['вес'].astype(str)

In [96]:
products_for_check = pd.concat([burn_product, dsmh_product])

In [97]:
merged = pd.merge(prices, products_for_check, on='Наименование товара', how='outer', indicator=True)

In [98]:
missing_values5 = merged.loc[ merged['_merge'] == 'right_only' ]

In [99]:
if missing_values5.empty == False:
    missing_values5.to_excel(path + '\missing_prices.xlsx', index=False)
else:
    print("Он пустой")

Он пустой


# РАБОТА С ФАЙЛОМ BURN

### адреса из справочника

In [100]:
need_addresses = addresses[['Исходный адрес', 'Адрес']]

In [101]:
burn_good_address = burn.merge(need_addresses, how="left", left_on='Адрес', right_on='Исходный адрес')

In [102]:
burn_good_address = burn_good_address.drop(columns=['Исходный адрес', 'Адрес_x'])

In [103]:
burn_good_address = burn_good_address.rename(columns={"Адрес_y": "Адрес"})

### регионы(субъекты) из справочника

In [104]:
burn_good_address = burn_good_address.merge(
    regions, how="left", left_on="Субъект РФ", right_on="Субъект РФ исходный", suffixes=('_left', '_right')
)

In [105]:
burn_good_address = burn_good_address.drop(columns=['Субъект РФ исходный', 'Субъект РФ_left'])

In [106]:
burn_good_address = burn_good_address.rename(columns={"Субъект РФ_right": "Субъект РФ"})

### изменить тип на дату и вычленить дату

In [107]:
burn_good_address['Дата операции'] = burn_good_address['Дата операции'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d')
)

In [108]:
burn_good_address['Месяц'] = burn_good_address['Дата операции'].dt.month

In [109]:
burn_good_address['Год'] = burn_good_address['Дата операции'].dt.year

### заменить бренды из справочника

In [110]:
burn_good_address = burn_good_address.merge(
    brands, how="left", left_on="Наименование товара", right_on="Бренд (исходный)", suffixes=('_left', '_right')
)

In [111]:
burn_good_address = burn_good_address.drop(columns=['Бренд (исходный)'])

### заменить граммовку

In [112]:
burn_good_address = burn_good_address.merge(
    weight, how="left", left_on="Наименование товара", right_on="Исходная граммовка", suffixes=('_left', '_right')
)

In [113]:
burn_good_address = burn_good_address.drop(columns=['Исходная граммовка'])

### Добавление колонки вес

In [114]:
burn_good_address['Вес'] = (burn_good_address['Граммовка']/1000)* burn_good_address['Объем товара (пачек)']

In [115]:
burn_good_address['Адрес'] = burn_good_address['Адрес'].fillna('не указан')

### цены из справочника

In [116]:
burn_good_address = burn_good_address.merge(prices, how="left", on='Наименование товара')

In [117]:
# должен быть ноль
burn_good_address['Цена'].isna().sum()

0

In [118]:
burn_good_address['Цена'] = burn_good_address['Цена'] * burn_good_address['Объем товара (пачек)']

### Группировка

In [145]:
burn_good_address.head(2)

,Дата операции,ИНН производителя,Наименование производителя,ИНН участника,Наименование участника,Код субъекта РФ,GTIN,ТН ВЭД,Наименование товара,Объем товара (пачек),МРЦ,Адрес,Субъект РФ,Месяц,Год,Бренд,Граммовка,Вес,Цена
0,2022-10-12,5047205725,"ООО ""БЁРН""",861602195158,ИП Мыкалов Владимир Николаевич,74.0,4650259250815,2403110000,Табак для кальяна BlackBurn Siberian Soda (Лим...,120,0,"г Челябинск, ул Труда, д 187, кв 1",Челябинская область,10,2022,BlackBurn,25,3.0,15840
1,2022-10-14,5047205725,"ООО ""БЁРН""",220400152969,ИП Авдиюк Ирина Анатольевна,22.0,4650190051472,2403110000,Табак для кальяна BlackBurn Chupa Graper (Вино...,83,0,"г Барнаул, ул Бабуркина, д 11А",Алтайский край,10,2022,BlackBurn,25,2.075,10956


In [146]:
burn_good = burn_good_address.groupby(['Год', 'Месяц', 'Субъект РФ', 'Бренд', 'Адрес']).agg({'Вес':'sum','Цена':'sum'}).reset_index()

In [147]:
burn_good.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 225080 entries, 0 to 225079
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Год         225080 non-null  int64 
 1   Месяц       225080 non-null  int64 
 2   Субъект РФ  225080 non-null  object
 3   Бренд       225080 non-null  object
 4   Адрес       225080 non-null  object
 5   Вес         225080 non-null  object
 6   Цена        225080 non-null  int64 
dtypes: int64(3), object(4)
memory usage: 12.0+ MB


# РАБОТА С ФАЙЛОМ MHDS

In [119]:
dsmh_copy = dsmh.copy(deep = True)

### Выделить месяц и год

In [120]:
dsmh_copy['Дата операции'] = dsmh_copy['мес'].apply(
    lambda x: datetime.strptime(x, '%Y-%m-%d')
)

In [121]:
dsmh_copy['Месяц'] = dsmh_copy['Дата операции'].dt.month

In [122]:
dsmh_copy['Год'] = dsmh_copy['Дата операции'].dt.year

### регион из справочника

In [123]:
dsmh_copy = dsmh_copy.merge(
    regions, how="left", left_on="рег", right_on="Субъект РФ исходный", suffixes=('_left', '_right')
)

In [124]:
dsmh_copy = dsmh_copy.drop(columns=['Субъект РФ исходный', 'рег'])

### адрес из справочника

In [125]:
need_addresses = addresses[['Исходный адрес', 'Адрес']]

In [126]:
dsmh_good_address = dsmh_copy.merge(need_addresses, how="left", left_on='адрес', right_on='Исходный адрес')

In [127]:
dsmh_good_address = dsmh_good_address.drop(columns=['Исходный адрес', 'адрес'])

### цены из справочника

In [128]:
dsmh_good_address['Наименование товара'] = dsmh_good_address['продукт'] + dsmh_good_address['вес'].astype(str)

In [129]:
dsmh_good_address = dsmh_good_address.merge(prices, how="left", on='Наименование товара')

In [130]:
dsmh_good_address['Цена'].isna().sum()

0

In [131]:
dsmh_good_address['Цена'] = dsmh_good_address['Цена'] * dsmh_good_address['колво']

### вес из кол-во и граммовки (у них вес называется)

In [132]:
dsmh_good_address['Новый вес'] = (dsmh_good_address['вес']/1000) * dsmh_good_address['колво']

In [134]:
dsmh_good_address = dsmh_good_address.drop(columns=['вес', 'колво'])

In [135]:
dsmh_good_address = dsmh_good_address.rename(columns = {'Новый вес': 'Вес'})

### бренд из справочника (менять не надо)

In [136]:
dsmh_good_address = dsmh_good_address.rename(columns={'бренд':'Бренд'})

### Группировка

In [137]:
dsmh_good_address = dsmh_good_address.groupby(['Год', 'Месяц', 'Субъект РФ', 'Бренд', 'Адрес']).agg({'Вес':'sum','Цена':'sum'}).reset_index()

In [138]:
dsmh_good_address.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 328257 entries, 0 to 328256
Data columns (total 7 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   Год         328257 non-null  int64  
 1   Месяц       328257 non-null  int64  
 2   Субъект РФ  328257 non-null  object 
 3   Бренд       328257 non-null  object 
 4   Адрес       328257 non-null  object 
 5   Вес         328257 non-null  float64
 6   Цена        328257 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 17.5+ MB


In [148]:
BBDSMH = pd.concat([dsmh_good_address, burn_good], ignore_index=True)

In [ ]:
# удалить ненужный адрес
BBDSMH = BBDSMH.loc[ BBDSMH['Адрес'] != 'г Санкт-Петербург, наб Пироговская, д 17 к 7' ]
BBDSMH = BBDSMH.loc[ BBDSMH['Адрес'] != 'г Санкт-Петербург, наб Пироговская, д 17 к 7 литера а' ]


In [ ]:
BBDSMH = BBDSMH.loc[ (BBDSMH['Бренд'] == 'DARKSIDE') | (BBDSMH['Бренд'] == 'MUST HAVE') | (BBDSMH['Бренд'] == 'BlackBurn') ]

In [159]:
BBDSMH.to_excel(r'D:\Analysis Burn\Википедия\Wikipedia.xlsx', index=False)